# Import your Libraries

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import numpy as np
import json
import requests
import os
import lusid
import cocoon

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()
scope = 'my_test_scope_6'

print ('LUSID Environment Initialised')
print ('LUSID API Version: ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version:  0.5.2885.0


# 1) Create Portfolios

## a) Import portfolio data

In [2]:
positions_file = pd.read_csv('./data/Sample_Positions.csv')
positions_file.head()

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900,141900,Lloyds Banking Group PLC,GB0008706128,870612,GBP
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000,2084000,Apple Inc,US0378331005,0865985,USD
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721,52916721,Amazon,US0231351067,906866,USD
3,Portfolio-X,05/02/2019,BAML,GBX,942354,200.50,188941977,188941977,Sainsbury,XS1139087933,B019KW7,GBP
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434,22038434,Tesla Inc,US88160R1014,BSJC712,USD


## b) Apply data transformations specific to your data

In [3]:
portfolios_data = pd.DataFrame(positions_file['FundCode'])
portfolios_data['display_name'] = portfolios_data['FundCode'].apply(lambda x: "Fund {}".format(x))
portfolios_data['created'] = "2010-10-09T08:00:00Z"
portfolios_data['base_currency'] = 'GBP'
portfolios_data['description'] = portfolios_data['FundCode'].apply(lambda x: "The fund with the fund code {}".format(x))
portfolios_data.head()

,FundCode,display_name,created,base_currency,description
0,Portfolio-Z,Fund Portfolio-Z,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-Z
1,Portfolio-Z,Fund Portfolio-Z,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-Z
2,Portfolio-Z,Fund Portfolio-Z,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-Z
3,Portfolio-X,Fund Portfolio-X,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-X
4,Portfolio-X,Fund Portfolio-X,2010-10-09T08:00:00Z,GBP,The fund with the fund code Portfolio-X


## c) Set your portfolio field mapping

In [4]:
portfolio_mapping_required = {
  'code': 'FundCode',
  'display_name': 'display_name',
  'created': 'created',
  'base_currency': 'base_currency'
}

portfolio_mapping_optional = {
  'description': 'description',
  'accounting_method': None
}

## d) Create your portfolios

In [5]:
responses = cocoon.create_portfolios_if_not_exist(
    client=client, 
    scope=scope, 
    data_frame=portfolios_data, 
    required_mapping=portfolio_mapping_required, 
    optional_mapping=portfolio_mapping_optional)

for portfolio_code, response in responses.items():
    prettyprint.portfolio_response(response)

# 2) Load your instrument universe

## a) Import instrument data

In [6]:
positions_file

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900,141900,Lloyds Banking Group PLC,GB0008706128,870612,GBP
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000,2084000,Apple Inc,US0378331005,0865985,USD
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721,52916721,Amazon,US0231351067,906866,USD
3,Portfolio-X,05/02/2019,BAML,GBX,942354,200.50,188941977,188941977,Sainsbury,XS1139087933,B019KW7,GBP
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434,22038434,Tesla Inc,US88160R1014,BSJC712,USD


## b) Apply data transformations specific to your data

In [7]:
instruments_data = pd.DataFrame(
    positions_file.loc[:, [
        'Local Currency Code', 
        'Security Description',
        'ISIN Security Identifier',
        'SEDOL Security Identifier']]
    )

## c) Set your instruments field mapping

In [8]:
instrument_mapping_required = {
  'name': 'Security Description'
}

instrument_identifier_mapping = {
  'identifier_mapping': {
    'ClientInternal': 'SEDOL Security Identifier',
    'Sedol': 'SEDOL Security Identifier',
    'Isin': 'ISIN Security Identifier'
  }
}

instrument_mapping_optional = {
  'definition': None
}

## d) Upsert your instruments

In [9]:
response = cocoon.upsert_instruments(
    client=client, 
    data_frame=instruments_data, 
    instrument_identifier_mapping=instrument_identifier_mapping, 
    instrument_mapping_required=instrument_mapping_required,
    instrument_mapping_optional=instrument_mapping_optional)

print (response)

{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://mikejmcgarry.lusid.com/api/schemas/entities/UpsertInstrumentsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'}],
 'values': {'Amazon': {'href': 'https://mikejmcgarry.lusid.com/api/instruments/LusidInstrumentId/LUID_RB16TCZW',
                       'identifiers': {'ClientInternal': '906866',
                                       'Isin': 'US0231351067',
                                       'LusidInstrumentId': 'LUID_RB16TCZW',
                                       'Sedol': '906866'},
                       'instrument_definition': None,
                       'links': None,
                       'lookthrough_portfolio': None,
                       'lusid_instrument_id': 'LUID_RB16TCZW',
                       'name': 'Amazon',
                       'properties': [],
                       'state': 'Active',
                       'version': {'as_at_date': date

# 3) Set your holdings

## a) Import holdings data

In [10]:
positions_file

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900,141900,Lloyds Banking Group PLC,GB0008706128,870612,GBP
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000,2084000,Apple Inc,US0378331005,0865985,USD
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721,52916721,Amazon,US0231351067,906866,USD
3,Portfolio-X,05/02/2019,BAML,GBX,942354,200.50,188941977,188941977,Sainsbury,XS1139087933,B019KW7,GBP
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434,22038434,Tesla Inc,US88160R1014,BSJC712,USD


## b) Apply data transformations specific to your data

In [11]:
positions_file['is_cash_with_currency'] = np.NaN
positions_file.head()

,FundCode,Effective Date,Prime Broker,Local Currency Code,Quantity,Local Price,Local Market Value,Base Market Value,Security Description,ISIN Security Identifier,SEDOL Security Identifier,Buy_Currency,is_cash_with_currency
0,Portfolio-Z,02/07/2019,UBS,GBP,2500,56.76,141900,141900,Lloyds Banking Group PLC,GB0008706128,870612,GBP,NaN
1,Portfolio-Z,01/01/2019,CITI,USD,10000,208.40,2084000,2084000,Apple Inc,US0378331005,0865985,USD,NaN
2,Portfolio-Z,01/12/2018,UBS,USD,26598,1989.50,52916721,52916721,Amazon,US0231351067,906866,USD,NaN
3,Portfolio-X,05/02/2019,BAML,GBX,942354,200.50,188941977,188941977,Sainsbury,XS1139087933,B019KW7,GBP,NaN
4,Portfolio-X,24/11/2018,MS,USD,95421,230.96,22038434,22038434,Tesla Inc,US88160R1014,BSJC712,USD,NaN


## c) Set your holdings field mapping

In [12]:
holding_mapping_required = {
  'portfolio_code': 'FundCode',
  'effective_date': 'Effective Date',
  'tax_lots.units': 'Quantity'
}

holding_identifier_mapping = {
  'identifier_mapping': {
      'Isin': 'ISIN Security Identifier',
      'Sedol': 'SEDOL Security Identifier'
  },
  'is_cash_with_currency': 'is_cash_with_currency'
}

holding_mapping_optional= {
  'tax_lots.cost.amount': None,
  'tax_lots.cost.currency': None,
  'tax_lots.portfolio_cost': None,
  'tax_lots.price': None,
  'tax_lots.purchase_date': None,
  'tax_lots.settlement_date': None
}

## d) Resolve to your instrument master

In [13]:
positions_file = cocoon.resolve_instruments(
    client=client,
    data_frame=positions_file,
    identifier_mapping=holding_identifier_mapping)

current_time = datetime.now(pytz.UTC)

positions_file.loc[
    positions_file['resolvable'] == False].to_csv(
        './data/UnResolved{}-{}-{}-{}-{}-{}.csv'.format(
            'Holdings',
            current_time.year,
            current_time.month,
            current_time.day,
            current_time.hour,
            current_time.minute))

positions_file['resolvable'].value_counts()

Up to row 0


True    5
Name: resolvable, dtype: int64

## e) Add your holdings

In [14]:
positions_file = positions_file.loc[positions_file['resolvable'] == True]

In [15]:
for effective_date in positions_file['Effective Date'].unique():
    
    positions_file_single_effective = positions_file.loc[
        positions_file['Effective Date'] == effective_date]
    
    print (positions_file_single_effective)

    responses = cocoon.load_file_multiple_portfolios(
        client=client, 
        scope=scope, 
        data_frame=positions_file_single_effective,
        mapping_required=holding_mapping_required, 
        mapping_optional=holding_mapping_optional,
        source='Client',
        file_type='holding')

    print (responses)

      FundCode Effective Date Prime Broker Local Currency Code  Quantity  \
0  Portfolio-Z     02/07/2019          UBS                 GBP      2500   

   Local Price  Local Market Value  Base Market Value  \
0        56.76              141900             141900   

       Security Description ISIN Security Identifier  \
0  Lloyds Banking Group PLC             GB0008706128   

  SEDOL Security Identifier Buy_Currency  is_cash_with_currency  resolvable  \
0                    870612          GBP                    NaN        True   

       foundWith LusidInstrumentId  \
0  [Isin, Sedol]     LUID_USQX4UWN   

                                             comment  
0  Uniquely resolved to an instrument in the secu...  
data types don't match for column is_cash_with_currency it is string in LUSID and float64 in file
Updated is_cash_with_currency to object
Check for missing holding properties complete


/home/jovyan/sample-notebooks/examples/use-cases/cocoon.py:564: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  data_type_update_map[data_type_lusid], copy=False)


{'Portfolio-Z': {'href': 'https://mikejmcgarry.lusid.com/api/transactionportfolios/my_test_scope_6/Portfolio-Z/holdings?asAt=2019-07-26T13%3A24%3A47.5181920%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://mikejmcgarry.lusid.com/api/portfolios/my_test_scope_6/Portfolio-Z?effectiveAt=2019-02-07T00%3A00%3A00.0000000%2B00%3A00&asAt=2019-07-26T13%3A24%3A47.5181920%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://mikejmcgarry.lusid.com/api/transactionportfolios/my_test_scope_6/Portfolio-Z/holdings?effectiveAt=2019-02-07T00%3A00%3A00.0000000%2B00%3A00&asAt=2019-07-26T13%3A24%3A47.5181920%2B00%3A00',
            'method': 'GET',
            'relation': 'Holdings'}],
 'version': {'as_at_date': datetime.datetime(2019, 7, 26, 13, 24, 47, 518192, tzinfo=tzlocal()),
             'effective_from': datetime.datetime(2019, 2, 7, 0, 0, tzinfo=tzlocal())}}}
      FundCode Effective Date Prime

In [16]:
r = client.transaction_portfolios.get_holdings(
    scope=scope,
    code="Portfolio-Z")

# 4) Add your transactions

## a) Import transactions data

## b) Apply data transformations specific to your data

## c) Set your transactions field mapping

In [ ]:
transaction_field_mapping_required = {
    "portfolio_code": ???,
    "transaction_id": ???,
    "transaction_type": ???,
    "transaction_date": ???,
    "settlement_date": ???,
    "units": ???,
    "transaction_price.price": ???,
    "total_consideration.amount": ???,
    "total_consideration.currency": ???,
    "transaction_currency": ???
    }

transaction_field_mapping_optional = {
    "exchange_rate": ???
}

transaction_identifier_mapping = {
    "identifier_mapping": {
        ???: ???
    },
    "is_cash_with_currency": ???
}

## d) Resolve to your instrument master

## e) Add your transactions